In [8]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [11]:
import getMTAdata as mta
df, links = mta.getAllDf(n_start=0, n_end=1)
filenames = [os.path.split(l)[1] for l in links]
print links
print filenames

getting data from the MTA website...
working on http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt
['http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt']
['turnstile_160402.txt']


In [12]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282


In [13]:
# or we can read from the csv file
# df = pandas.read_csv("Saturday, April 02, 2016") 
df.columns = ['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','IN','OUT']
df = df[['UNIT', 'SCP', 'DATE', 'TIME', "IN", "OUT"]]
df.head(5)

,UNIT,SCP,DATE,TIME,IN,OUT
0,R051,02-00-00,03/26/2016,00:00:00,5595746,1893277
1,R051,02-00-00,03/26/2016,04:00:00,5595746,1893282
2,R051,02-00-00,03/26/2016,08:00:00,5595746,1893282
3,R051,02-00-00,03/26/2016,12:00:00,5595746,1893282
4,R051,02-00-00,03/26/2016,16:00:00,5595746,1893282


In [14]:
# Create a coordinates look up table and add a column

geocode = pandas.read_csv('geocoded.csv', header=None)
geocode = geocode.drop_duplicates(0)
geocode = geocode[[0,5,6]]
geocode.columns = ['UNIT', 'LAT', 'LON']
geocode_mapping = {row.values[0]:[row[1], row[2]] for index,row in geocode.iterrows()}

def map(unit):
    try:
        return geocode_mapping[unit]
    except:
        return np.nan

df['COORD'] = df['UNIT'].apply(map)
df['LAT'] = df['COORD'].apply(lambda x: x[0])
df['LON'] = df['COORD'].apply(lambda x: x[1])

In [15]:
df.head(5)

,UNIT,SCP,DATE,TIME,IN,OUT,COORD,LAT,LON
0,R051,02-00-00,03/26/2016,00:00:00,5595746,1893277,"[40.762796, -73.967686]",40.762796,-73.967686
1,R051,02-00-00,03/26/2016,04:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686
2,R051,02-00-00,03/26/2016,08:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686
3,R051,02-00-00,03/26/2016,12:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686
4,R051,02-00-00,03/26/2016,16:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686


In [16]:
#Add the times as datetime objects

import datetime
df["DATETIME"] = df["DATE"]+ ' ' + df["TIME"]
df["DATETIME"] = pandas.to_datetime(df["DATETIME"])

# uncomment below if we want the day to be int
# df["DAY"] = df.DATETIME.apply(lambda x: int(x.weekday()))

In [17]:
def classify_day(time):
    if time.isoweekday() in range(1,6):
        return "Weekday"
    else:
        return "Weekend"

df["DAY"] = df['DATETIME'].apply(classify_day)

In [18]:
df.head()

,UNIT,SCP,DATE,TIME,IN,OUT,COORD,LAT,LON,DATETIME,DAY
0,R051,02-00-00,03/26/2016,00:00:00,5595746,1893277,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 00:00:00,Weekend
1,R051,02-00-00,03/26/2016,04:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 04:00:00,Weekend
2,R051,02-00-00,03/26/2016,08:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 08:00:00,Weekend
3,R051,02-00-00,03/26/2016,12:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 12:00:00,Weekend
4,R051,02-00-00,03/26/2016,16:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 16:00:00,Weekend


In [19]:
def classify_time(time):
    if 5 <= time.hour <= 9:
        return "Morning"
    elif 17 < time.hour < 22:
        return "Evening"
    else:
        return None
df["M_E"] = df['DATETIME'].apply(classify_time)

In [20]:
df.head()

,UNIT,SCP,DATE,TIME,IN,OUT,COORD,LAT,LON,DATETIME,DAY,M_E
0,R051,02-00-00,03/26/2016,00:00:00,5595746,1893277,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 00:00:00,Weekend,None
1,R051,02-00-00,03/26/2016,04:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 04:00:00,Weekend,None
2,R051,02-00-00,03/26/2016,08:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 08:00:00,Weekend,Morning
3,R051,02-00-00,03/26/2016,12:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 12:00:00,Weekend,None
4,R051,02-00-00,03/26/2016,16:00:00,5595746,1893282,"[40.762796, -73.967686]",40.762796,-73.967686,2016-03-26 16:00:00,Weekend,None


In [48]:
%%time

# make a group reduce function to apply to df.groupby
def group_manipulation(df):
    reduce_df = pandas.Series()
    IN = df.IN.values; OUT = df.OUT.values
    IN = IN[1:] - IN[:-1] # convert from cumulative
    OUT = OUT[1:] - OUT[:-1]
    mask = (IN >= 0) & (IN < 1e4) & (OUT >= 0) & (OUT < 1e4)
    reduce_df['UNIT'] = df.iloc[0]['UNIT']
    reduce_df['IN'] = IN[mask].sum()
    reduce_df['OUT'] = OUT[mask].sum()
    reduce_df['LAT'] = df.iloc[0]['LAT']
    reduce_df['LON'] = df.iloc[0]['LON']
    return reduce_df

target_DAY = ['Weekday', 'Weekend']
target_M_E = ['Morning', 'Evening']

# filter df to target day and timeslot
df = df[df['DAY'] == target_DAY]
df = df[df['M_E'] == target_M_E]

masterDF = df.groupby('UNIT', as_index=False).apply(group_manipulation)

CPU times: user 1.1 s, sys: 32.7 ms, total: 1.13 s
Wall time: 1.11 s


In [54]:
print "sum of IN",  masterDF.IN.sum()
print "sum of OUT", masterDF.OUT.sum()
ratio = float(masterDF.OUT.sum()) / masterDF.IN.sum()
print "Total OUT is {:%} of total IN".format(ratio)
adjust = 1. / ratio
print "an ajustment parameter should be applied: ", adjust


def model_outflux(OUT, adjust=1.33):
    """model to adjust outflux for the purpose of mass conservation"""
    return OUT * adjust

sum of IN 24298733
sum of OUT 17906609
Total OUT is 73.693591% of total IN
an ajustment parameter should be applied:  1.3569700997


In [57]:
masterDF['OUTPRIME'] = masterDF['OUT'].apply(
    lambda x: model_outflux(x, adjust)
)
masterDF['FLUX'] = (masterDF.IN - masterDF.OUTPRIME)
masterDF.head()

,UNIT,IN,OUT,LAT,LON,OUTPRIME,FLUX
0,R001,121664,123031,40.703082,-74.012983,166949.388336,-45285.388336
1,R003,6660,3798,40.689945,-73.872564,5153.772439,1506.227561
2,R004,16872,15319,40.691320,-73.867135,20787.424957,-3915.424957
3,R005,16468,8112,40.692304,-73.860151,11007.741449,5460.258551
4,R006,20230,9054,40.693866,-73.851568,12286.007283,7943.992717


In [58]:
filename_group = ''.join([f.split('.')[0] for f in filenames]) + '_group.csv'
print 'saving', filename_group
masterDF.to_csv(filename_group, index_label=None)

saving turnstile_160402_group.csv


In [52]:
masterDF.head()

,UNIT,IN,OUT,LAT,LON,OUTPRIME
0,R001,121664,123031,40.703082,-74.012983,166949.388336
1,R003,6660,3798,40.689945,-73.872564,5153.772439
2,R004,16872,15319,40.691320,-73.867135,20787.424957
3,R005,16468,8112,40.692304,-73.860151,11007.741449
4,R006,20230,9054,40.693866,-73.851568,12286.007283


In [ ]:
%%time
# --- this is the same as the above cell to group UNIT
# --- but it is looping instead of applying group function

masterDF = pandas.DataFrame(columns=['UNIT', 'IN', 'OUT', 'COORDS'])
DAY = 'Weekday'
M_E = "Evening"
for unit, group in df.groupby(['UNIT']):
    
    # Filter for weekday mornings
    day = group[group.DAY == DAY]
    timeOfDay = day[day.M_E == M_E]
    
    IN = timeOfDay.IN.values; OUT = timeOfDay.OUT.values
    IN = IN[1:] - IN[:-1] # convert from cumulative
    OUT = OUT[1:] - OUT[:-1]
    mask = (IN >= 0) & (IN < 1e4) & (OUT >= 0) & (OUT < 1e4)
    masterDF.loc[len(masterDF)] = (unit, IN[mask].sum(), 
                                   OUT[mask].sum(), group['COORD'].iloc[0])

    print "%s STATION FINISHED"%unit

masterDF.to_csv("SaturdayApril022016-%s-%s"%(DAY, M_E))

In [53]:
masterDF.head()

,UNIT,IN,OUT,LAT,LON,OUTPRIME
0,R001,121664,123031,40.703082,-74.012983,166949.388336
1,R003,6660,3798,40.689945,-73.872564,5153.772439
2,R004,16872,15319,40.691320,-73.867135,20787.424957
3,R005,16468,8112,40.692304,-73.860151,11007.741449
4,R006,20230,9054,40.693866,-73.851568,12286.007283


In [ ]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
plt.figure(figsize=(12,12))

for i, masterDF in enumerate([pandas.read_csv("SaturdayApril022016-Weekday-Mornings"),
                pandas.read_csv("SaturdayApril022016-Weekday-Evening")]):
    
    plt.subplot(2,1,i)
    my_map = Basemap(projection='merc', resolution = 'l',
        llcrnrlon=-74.1, llcrnrlat=40.7,
        urcrnrlon=-73.9, urcrnrlat=40.8)

    #    llcrnrlon=-74.2, llcrnrlat=40.5,
    #    urcrnrlon=-73.7, urcrnrlat=41)

    my_map.drawcoastlines()
    my_map.drawcountries()
    #my_map.fillcontinents(color='coral')
    my_map.drawmapboundary()

    #my_map.drawmeridians(np.arange(0, 360, 30))
    #my_map.drawparallels(np.arange(-90, 90, 30))

    masterDF = masterDF.dropna(subset=['COORDS', 'IN', 'OUT'])

    mycolmap = plt.get_cmap("seismic")
    colors = masterDF['IN'] - masterDF['OUT']
    sizes = 60*(masterDF['IN'] + masterDF['OUT']) / (masterDF['IN'] + masterDF['OUT']).max()
    coords = list(masterDF['COORDS'].astype(list))
    lon = [coord[0] for coord in coords]
    lat = [coord[1] for coord in coords]
    x,y = my_map(lat, lon)
    plt.scatter(x, y, c=colors, cmap=mycolmap, s=sizes)
    
plt.colorbar()
plt.show()

